In [1]:
pip install --upgrade --no-cache-dir sports_prediction_framework

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 263.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 276.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
  Attempting uninstall: sports_prediction_framework
    Found existing installation: sports_prediction_framework 0.1.1
    Uninstalling sports_prediction_framework-0.1.1:
      Successfully uninstalled sports_prediction_framework-0.1.1


In [10]:
from sports_prediction_framework.datawrapper.SportType import SportType
from sports_prediction_framework.dataloader.DataLoader import DataLoader
from sports_prediction_framework.datawrapper.DataHandler import DataHandler
import pandas as pd


from google.colab import files
uploaded = files.upload()

df = pd.read_parquet('data.parquet')


handler = DataHandler(df)
dw = SportType.FOOTBALL.get_wrapper()(handler)
print(dw)

Saving data.parquet to data (3).parquet
            Date           Home            Away  HS  AS  WDL  odds_1  odds_X  \
0     2004-01-21  Bayern Munich    Hamburger SV   3   0    1    1.39    4.00   
1     2004-01-22      Wolfsburg        Dortmund   1   2    2    1.83    3.25   
2     2004-01-22       Nurnberg  Kaiserslautern   1   3    2    2.10    3.25   
3     2004-01-22       Nurnberg  Kaiserslautern   1   3    2    2.00     NaN   
4     2004-01-22          Mainz       Stuttgart   2   3    2    2.79    3.25   
...          ...            ...             ...  ..  ..  ...     ...     ...   
10301 2022-02-13   Union Berlin        Dortmund   0   3    2    3.80    3.60   
10302 2022-02-13   Union Berlin        Dortmund   0   3    2    3.50    3.50   
10303        NaT  Hertha Berlin        Duisburg   2   0    1    1.62    3.60   
10304        NaT  Hertha Berlin        Duisburg   2   0    1    1.05   10.00   
10305        NaT  Hertha Berlin        Duisburg   2   0    1     NaN     NaN   


In [11]:
from sports_prediction_framework.transformer.Transformer import *
#Applies basic transformations to the dataframe.
t = Transformer()
dw = t.transform(dw)
print(dw)

            Date           Home            Away  HID  AID  HS  AS  WDL  \
0     2004-01-21  Bayern Munich    Hamburger SV    5   17   3   0    1   
1     2004-01-22      Wolfsburg        Dortmund   35    9   1   2    2   
2     2004-01-22       Nurnberg  Kaiserslautern   27   24   1   3    2   
3     2004-01-22       Nurnberg  Kaiserslautern   27   24   1   3    2   
4     2004-01-22          Mainz       Stuttgart   26   32   2   3    2   
...          ...            ...             ...  ...  ...  ..  ..  ...   
10301 2022-02-13   Union Berlin        Dortmund   33    9   0   3    2   
10302 2022-02-13   Union Berlin        Dortmund   33    9   0   3    2   
10303        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   
10304        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   
10305        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   

       odds_1  odds_X  odds_2  
0        1.39    4.00    6.50  
1        1.83    3.25    3.75  
2        2.10  

In [12]:
from sports_prediction_framework.transformer.Scope import *
from sports_prediction_framework.transformer.DataSelector import *

relevant_scope = [WindowSelector(ScopeExpander(dw, {
    'col': 'Season', 'start': 2004, 'max': 2008, 'size': 1, 'stride': 2
}))]
prediction_scope = [WindowSelector(ScopeExpander(dw, {
    'col': 'Season', 'start': 2004, 'max': 2008, 'size': 1, 'stride': 2
}))]
scope = DataSelector(relevant_scope, prediction_scope)

In [13]:
from sports_prediction_framework.model.FlatModel import FlatModel
from sports_prediction_framework.learner.Learner import Learner, UpdatingLearner, Tester, Trainer

params = {
    'embed_dim': 32,
    'out_dim': 3,
    'n_dense': 4,
    'dense_dim': 64,
    'architecture_type': 'rectangle',
    'batch_size': 64,
}
flat = FlatModel(params)

#Setup learner and updating learner
l1 = Learner(Trainer(flat), Tester(flat), scope)
l = UpdatingLearner(Trainer(flat), Tester(flat), scope, [l1])

In [15]:
prob = l.compute(dw)
#Predictions stored in 0,1,2 columns.
print(prob)

            Date           Home            Away  HID  AID  HS  AS  WDL  \
0     2004-01-21  Bayern Munich    Hamburger SV    5   17   3   0    1   
1     2004-01-22      Wolfsburg        Dortmund   35    9   1   2    2   
2     2004-01-22       Nurnberg  Kaiserslautern   27   24   1   3    2   
3     2004-01-22       Nurnberg  Kaiserslautern   27   24   1   3    2   
4     2004-01-22          Mainz       Stuttgart   26   32   2   3    2   
...          ...            ...             ...  ...  ...  ..  ..  ...   
10301 2022-02-13   Union Berlin        Dortmund   33    9   0   3    2   
10302 2022-02-13   Union Berlin        Dortmund   33    9   0   3    2   
10303        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   
10304        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   
10305        NaT  Hertha Berlin        Duisburg   20   10   2   0    1   

       odds_1  odds_X  odds_2         1         0         2  
0        1.39    4.00    6.50  0.564068  0.276957